# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data_df = pd.read_csv("../WeatherPy/weather_data")
weather_data_df.dropna()
weather_data_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp
0,makat,47.65,53.35,28.76,92,0,8.50,28.76
1,kapaa,22.08,-159.32,70.39,83,1,6.93,71.60
2,castro,-24.79,-50.01,67.51,98,100,3.49,67.51
3,nantucket,41.28,-70.10,39.11,69,90,6.93,39.99
4,abu samrah,35.30,37.18,45.01,64,0,5.97,45.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)


In [6]:
# Store latitude and longitude in locations
locations = weather_data_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = weather_data_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Narrow down locations based on max temp, wind speed, and humidity
ideal_max_temp = weather_data_df.loc[weather_data_df['Max Temp'] < 80]
ideal_temp = ideal_max_temp.loc[ideal_max_temp['Max Temp'] > 70]
ideal_wind = ideal_temp.loc[ideal_temp['Wind Speed'] < 10 ]
ideal_places = ideal_wind.loc[ideal_wind['Humidity'] < 65]
ideal_places.count()

City           21
Latitude       21
Longitude      21
Temperature    21
Humidity       21
Cloudiness     21
Wind Speed     21
Max Temp       21
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:

#create hotel_df
hotel_df = ideal_places[['City', 'Latitude', 'Longitude']]
hotel_df["Hotel Name"] = ""
hotel_df["Country"] = ""
hotel_df.head()

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,City,Latitude,Longitude,Hotel Name,Country
72,tessalit,20.20,1.01,,
94,kita,13.03,-9.49,,
148,kidal,18.44,1.41,,
152,pascagoula,30.37,-88.56,,
236,salalah,17.02,54.09,,


In [9]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request
    name_address = requests.get(base_url, params=params).json()

    #Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Country"] = name_address["results"][0]["vicinity"]
        
    except (KeyError, IndexError):
    
        print("Missing field/result... skipping.")

Missing field/result... skipping.


/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [10]:
hotel_df


,City,Latitude,Longitude,Hotel Name,Country
72,tessalit,20.20,1.01,,
94,kita,13.03,-9.49,Hôtel Chat Rouge,Kita
148,kidal,18.44,1.41,,
152,pascagoula,30.37,-88.56,Hard Rock Hotel and Casino Biloxi,"777 Beach Blvd, Biloxi"
236,salalah,17.02,54.09,Salalah Rotana Resort,"Hawana Salalah Taqah Road, Salalah"
247,bria,6.54,21.99,,
294,mmabatho,-25.85,25.63,Peermont Walmont Hotel at Mmabatho Palms,"1 Nelson Mandela Dr, Mmabatho Sun Hotel, Mmabatho"
312,birao,10.28,22.79,,
315,santona,43.44,-3.46,Hotel ILUNION Bilbao,"Rodríguez Arias Kalea, 66, Bilbo"
389,mana,5.66,-53.78,Hôtel Amazonia du Fleuve,"20 Rue Thiers, Saint-Laurent-du-Maroni"


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

# Display Map

IndexError: list index out of range